# Single Heart Rate Monitor Example

This is a minimal demonstration of how to set up and use Bleak-FSM.

This examples uses Bluetooth Low Energy (BLE) Heart Rate service.
+ It must support Bluetooth, not just ANT+
+ The GATT Characteristic required is: `00002a37-0000-1000-8000-00805f9b34fb`
+ You may use the [`bluetooth_dissect.py` tool](https://github.com/tensorturtle/bluetooth_dissect) to check if your device supports that GATT Characteristic.

Example compatible devices:
+ Magene H603
+ Wahoo TICKR & TICKR Fit
+ WHOOP 4.0 (Broadcast Heart Rate ON)
+ and many more advertised as "supports Bluetooth Heart Rate"

If you don't have access to a BLE Heart Rate device or want to a different BLE GATT characteristic, see the [Migration Guide](migration_guide.ipynb)


## How to run this Notebook

+ Make sure your computer's bluetooth adapter is powered on.
+ At first, I recommend stepping through each cell, reading the code and explanations.
+ You will need to set your Heart Rate monitor's address the first time.
+ At any time, `print(model.state)` to confirm which state you're in. This will help you figure out what you can do from there.

## Using a `with` block Context Manager

The present notebook shows you the most detailed API. Bleak-FSM also offers a built-in context manager that you can optionally use to simplify your code.

[See the context manager example (which is based on this notebook)](examples/context_manager.ipynb)

# Setup

In [ ]:
import asyncio

In [ ]:
from bleak_fsm import BleakModel

In [ ]:
model = BleakModel(connection_timeout=20)

Toggle this `USE_PYCYCLING` flag to see how this library can accomodate using standard, raw BLeakClient or Pycycling-wrapped BleakClient.

In [ ]:
USE_PYCYCLING = False

In [ ]:
if USE_PYCYCLING:
    from pycycling.heart_rate_service import HeartRateService
    model.wrap = lambda client: HeartRateService(client)
    model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
    model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
    def handle_hr_measurement(value):
        print("Using Pycycling wrapper around BleakClient")
        print(f"Heart Rate: {value}")
    model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)
else:
    HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"

    def handle_hr_measurement(sender, data):
        print("Using raw BleakClient")
        print(f"Data received from {sender}: {data}")
        flag = data[0]
        heart_rate = data[1]
        print(f"Heart Rate: {heart_rate} beats per minute")
    model.enable_notifications = lambda client: client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)
    model.disable_notifications = lambda client: client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)

# Usage

Since we're in a Jupyter notebook, we need to use `create_task()` instead of `get_event_loop().run_until_complete()` to use the existing event loop which Jupyter runs on.

The `BleakModel` class represents this system's bluetooth adapter. This library currently supports one bluetooth adapter. We call the `start_scan()` class method.

In [ ]:
await BleakModel.start_scan()

In [ ]:
async def print_as_we_go(max_devices=30):
    for i in range(40):
        if len(BleakModel.bt_devices) >= max_devices:
            print(f"{len(BleakModel.bt_devices)} devices found. Stopping scan.")
            await model.stop_scan()
            break
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

In [ ]:
await BleakModel.stop_scan()

In [ ]:
BleakModel.bt_devices

Copy-and-paste the key corresponding to the device that you want to connect to:

In [ ]:
target_address = "EF:E4:F6:7D:6C:F6"

Or search for the name of the device from the values and retrieve its key.

While most consumer heart rate monitors have a name, this name might be not unique or missing.

In [ ]:
target_name = "WHOOPDEDOO"

target_address = ""
for address, (ble_device, advertisement_data) in BleakModel.bt_devices.items():
    if ble_device.name == target_name:
        target_address = address
        print(f"Found {target_name} at {target_address}")

Anyhow, we will attempt to connect to the following address:

In [ ]:
print(target_address)

In [ ]:
await model.set_target(target_address)

 When you provide a target address to the `model`, its state changes to `TargetSet`

In [ ]:
model.state

From `TargetSet` state, you may now attempt to connect to the device.

By default, if it has been over 120 seconds since the last scan, if `auto_rescan=True`, the model will automatically re-scan and then attempt the connection.
These parameters may be changed in the `BleakModule` instantiation above.

In [ ]:
success = await model.connect()

if success:
    print("Connected to device")

Note how the above cell returns `True`. All transition methods on `model` returns True if the transition was successful. Therefore you may explicitly define happy-path and exception-path in different ways.

For debugging,
```python
assert await model.connect(), "Model failed to connect"
```

Or, more properly check the return value
```python
success = await model.connect()
if not succcess:
    await model.clean_up() # this catch-all method brings the model back to "Init"
    raise Exception("Failed to connect")
```

Or, we can check the `.state` afterwards, like so:

In [ ]:
if model.state != "Connected":
    raise Exception("Failed to connect. If the device was not properly disconnected last time, restart either the device or bluetooth adapter")

In order to receive/send actual data from/to the connected device, we need to start a stream.

This will run forever until `model.disconnect()` is called, so for now let's sleep for several seconds and then call that.

In [ ]:
await model.stream()

In [ ]:
await asyncio.sleep(10)

Bleak-FSM encapsulates a kind of bug in Bleak, where it is not possible to re-use a connected client. 

That is, we shouldn't step back to "Connected" state from "Streaming", because we can't go from "Connected" back to "Streaming".

Instead we need to go back two steps to "TargetSet", re-establish a connection, and then we can stream again.

Bleak-FSM enforces this behavior for your own safety.
There exists not method for you to even try to go from "Streaming" to "Connected".


In [ ]:
await model.disconnect()

Whenever the application terminates, or otherwise no longer needs to communicate with the bluetooth device, it should always call `disconnect()` on the model. The model state should always e either "TargetSet" or "Init".

In [ ]:
model.state

In [ ]:
await model.unset_target()

In [ ]:
model.state

For convenience, you may call `await model.clean_up()` from any state to come back to "Init"

In [ ]:
await model.clean_up()

For even more convenience, you may call the class's `clean_up_all()` classmethod to disconnect all devices:

In [ ]:
await BleakModel.clean_up_all()